In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
ps = PorterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pickle 

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def ExploratoryDataAnalysis(Inputdataframe):           
        ColumnsList = []
        NullList = []
        NAList = []
        CountList = []    
        #Inputdataframe.drop('id', axis=1, inplace=True)
        for col in Inputdataframe.columns:        
            if(col == 'id' or col == 'comment_text'):
                continue;
            CountList.append(Inputdataframe[Inputdataframe[col] == 1].shape[0])
            ColumnsList.append(col)
            NullList.append(Inputdataframe[Inputdataframe[col] == 1]["comment_text"].isnull().sum())
            NAList.append(Inputdataframe[Inputdataframe[col] == 1]["comment_text"].isna().sum())
        df = pd.DataFrame(list(zip(ColumnsList,NullList,NAList,CountList)), 
                   columns =['Column Name','Null Count','NA Count','Record Count']) 
        return df.transpose(),"Total Records : {}".format(Inputdataframe.shape[0])        

In [0]:
def GetCleanedData(dataset):
    stop_words= stopwords.words("english")    
    Remove_words = [ 'no', 'nor', 'not','don', "don't", 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    for item in Remove_words: 
        stop_words.remove(item) 
        
    for i in range(dataset.shape[0]):
        comment_text = dataset.iloc[i, 1]

        # remove non alphabatic characters
        comment_text = re.sub('[^A-Za-z]', ' ', comment_text)

        # make words lowercase, because Go and go will be considered as two words
        comment_text = comment_text.lower()

        # tokenising
        tokenized_comments = word_tokenize(comment_text)

        #Remvoing stop words and Stemming
        comment_processed = []
        for item in tokenized_comments:
            if item not in (stop_words):                           
              if(len(item) <= 40):
                strstem = ps.stem(item) 
                comment_processed.append(strstem)
        dataset.iloc[i, 1] = " ".join(comment_processed)  
        print(i)
    return dataset

In [0]:
def GetDataSample(dataset):
  GetCleanedData(dataset)
  categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
  x_train, x_test, y_train, y_test = train_test_split(dataset["comment_text"], dataset[categories], train_size=0.95, shuffle=True, random_state=1)
  return x_train, x_test, y_train, y_test

In [0]:
def NaiveBayesModel(dataset):
    x_train, x_test, y_train, y_test = GetDataSample(dataset)
    
    # ngram_range = 1,2 = bigram model
    NB_Classifier = Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features = 50000)),
                ('clf', OneVsRestClassifier(MultinomialNB(fit_prior=True, class_prior=None))),
            ])
    
    NB_Classifier.fit(x_train, y_train)
    prediction = NB_Classifier.predict(x_test) 
    print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
      
    return NB_Classifier

In [0]:
dataset = pd.read_csv('/content/gdrive/My Drive/DataScience/ProjectWork/train.csv')


In [0]:
NB_Model  = NaiveBayesModel(dataset)

In [0]:
def predict(comment,model):
  categories = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
  probs = model.predict_proba([comment])[0]
  for (prob, category) in zip(probs, categories): 
    print('{} : {}%'.format(category, (round(prob,2)*100) ))

In [0]:
predict("he is a good boy. he loves to talk shit.",NB_Model)

In [0]:
#save the model to a file
with open('NB_Model_lat.pkl', 'wb') as f:
    pickle.dump(NB_Model, f)
file = drive.CreateFile({'title' : 'NB_Model_lat.pkl'})
file.SetContentFile('NB_Model_lat.pkl')
file.Upload() 